In [1]:
import folium
from src.apikeys import MONGO
from src import process_tweets
from pymongo import MongoClient
from src.model import TweetPredictor
import pandas as pd

In [2]:
tweet_processor = process_tweets.TweetProcessor('../models/emoticon_lr_model.pk')


In [3]:
client = MongoClient('mongodb://{}:{}@localhost:27017'.format(MONGO.USER, MONGO.PASSWORD))

In [4]:
database = client['twitter_database']

In [5]:
database.collection_names()

['test',
 'trends_2017_10_18_9',
 'trends_2017_10_16_17',
 'trends_2017_10_17_18',
 'broncos_raiders',
 'available_trends',
 'test_method',
 'trends_2017_10_17_19',
 'patriots_panthers',
 'trends_2017_10_17_17',
 'politics',
 'roll_tide']

In [6]:
table = database['patriots_panthers']

In [7]:
df = tweet_processor.process_database(table, topics=['patriots'])

In [8]:
df.head(10)

,text,state,date,sentiment,time_bin
0,RT @Patriots: LET'S GO. https://t.co/Ot4IjanSa1,Virginia,2017-10-01 16:54:59+00:00,0.754117,0
1,@Patriots #PatsNation #NewEnglandPatriots #Do...,Florida,2017-10-01 16:54:59+00:00,0.754117,0
2,RT @Patriots: #TFW you hear #boomtower is back...,North Carolina,2017-10-01 16:55:00+00:00,0.754117,0
3,Theres always some extra hate in me whenever t...,North Carolina,2017-10-01 16:55:00+00:00,0.180915,0
4,RT @MikeReiss: Videoboard shows Jerod Mayo ope...,Washington,2017-10-01 16:55:01+00:00,0.640178,0
5,RT @Patriots: LET'S GO. https://t.co/Ot4IjanSa1,Massachusetts,2017-10-01 16:55:02+00:00,0.754117,0
6,Watch Carolina Panthers vs New England Patriot...,Nevada,2017-10-01 16:55:03+00:00,0.763551,0
7,Carolina Panthers VS New England Patriots Live...,None,2017-10-01 16:55:04+00:00,0.763551,0
8,RT @Deacon_1971: POLL: Which POTUS loves Ameri...,Wisconsin,2017-10-01 16:55:04+00:00,0.640178,0
9,GAMETIME: Your New England Patriots look to im...,None,2017-10-01 16:55:03+00:00,0.763680,0


In [9]:
from src.visualization import visualize_sentiment
from src.visualization import visualize_count
my_map = visualize_sentiment(df)
my_map

In [11]:
my_map.save('../website/maps/patriots_sentiment.html')

In [53]:
import numpy as np
a = df.groupby('state')['sentiment'].agg(['mean','std','count'])
a['mean_dev'] = a.apply(lambda row: row['std']/np.sqrt(row['count']),axis=1)
a[['mean','mean_dev']].head(50)

,mean,mean_dev
state,,
Alabama,0.585976,0.008155
Alaska,0.603109,0.020606
Arizona,0.614509,0.007538
Arkansas,0.626332,0.016440
California,0.598246,0.003520
Colorado,0.585547,0.009064
Connecticut,0.576652,0.005627
Delaware,0.601214,0.005925
District of Columbia,0.590709,0.014777


In [27]:
my_map, avg_sentiment = visualize_count(df[df.sentiment<.3])
print(avg_sentiment)
my_map.save('../website/maps/patriots_count.html')
my_map

0.17911979737423891


In [26]:
for i in df[df.state=='Massachusetts'][['text','sentiment']].values:
    print(i[0],i[1])

RT @Patriots: LET'S GO. https://t.co/Ot4IjanSa1 0.7541169002367804
Intrigante saber como responderá la defensiva de #patriots hoy vs #panters. estan en el sótano de la NFL 0.39671019079976555
Intrigante saber como responderá la defensiva de #patriots hoy vs #panters. estan en el sótano de la NFL 0.39671019079976555
#Patriots time!!!!! #GoPats #PatriotsNation 0.5044144893581977
Moss names Brady the best QB of all time - ESPN Video #Patriots  https://t.co/DY3oKl4MLp https://t.co/7KBzd7EGxU 0.8000890942853068
RT @NEPatsNation12: GAMETIME: Your New England Patriots look to improve their record to 3-1 against the Carolina Panthers! https://t.co/dJV… 0.7999007944711222
MATT .@mattbare23 Sorry  your request came late to me : I already put out the #Patriots as my best play if they win… https://t.co/B1jHcbVGzN 0.5120278809901939
RT @Patriots: It's game day in Foxboro! https://t.co/2tQ13eG8Xk 0.6742112145104175
RT @MarkDanielsPJ: Looks like the Patriots will stand together for the anthem 0.54063

Go Patriots! https://t.co/H2CwAd1BGx 0.6955919972793633
Develin! Again? Again! I want this more  often! #Patriots 0.2123573047861984
RT @alyssabflowers: Football Sunday is my favorite day of the week. 

Like for Panthers. RT for Patriots. 0.5116104083554831
It’s nice to see Devlin be used as something other than a blocking FB #CARvNE #NFL #Patriots 0.7540411833644257
RT @NFL: .@Patriots INT!

@Mac_BZ intercepts Cam on the deep throw. #GoPats #CARvsNE https://t.co/9LEUK2piXV 0.7554083706875145
RT @Patriots: Picked off by @mac_bz! https://t.co/3fR8phhfhU 0.7638104706263552
Develin. #Patriots #CARvsNE https://t.co/MAkwc83KcQ 0.6955919972793633
End of 1st: Patriots 3, Panthers 3 0.39671019079976555
Come on White, get us that touchdown! #GoPats @Patriots #Patriots 0.5334250184090665
RT @Patriots: Picked off by @mac_bz! https://t.co/3fR8phhfhU 0.7638104706263552
Game day #football #Sunday #gopatriots #sundayfootball  #patriots 🏈 @ Minerva Pizzeria and Slice… https://t.co/ifNzzkBjOo 0.6831765

No game is a given for this team this year, Jets &amp; Dolphins included #patriots defense is that bad 0.103508748930885
Patriots defense as of right now is TERRIBLE. Miscommunication galore. I mean it's been unacceptable. #NFL 0.40086387714198113
It's Good!!!! #Patriots 0.6269886683238431
Thought that ball was gonna be too shallow, but 58 yards later, 3 more pts on the board for #Patriots #PatsNation #CARvsNE 0.43896426629102
#PATRIOTS ERIC ROWE gets an  (  F ) from me on a Play that cost the .@Patriots dearly  the .@PANTHERS SCORE A TOUCH… https://t.co/9vaW7hcpAa 0.7638104706263552
Headed to @boston25 FB page to test your #Patriots trivia and give away a free hat from @47 - everyone likes free hats. 0.6401778316553294
Patriots 16 - 17 Panthers 0.39671019079976555
What pathetic defense from the Carolina #Panthers. Gifted 3 points to the #Patriots. 0.39671019079976555
SCORE UPDATE: Patriots 16, Panthers 17 at Halftime. 0.39671019079976555
Gostkowski good from 58, and first half ends wi

No gain for you! #CARvsNE #GoPats @Patriots #Patriots https://t.co/vF5vBtKdCZ 0.7541169002367804
RT @spot_patriots: Well, I guess that halftime speech kinda worked. #Panthers fumble on their first drive #Patriots recover!!! 0.6401778316553294
RT @spot_patriots: Well, I guess that halftime speech kinda worked. #Panthers fumble on their first drive #Patriots recover!!! 0.6401778316553294
Is this was watching football feels like for the other 31 NFL teams all these years? #Patriots 0.6624321983576307
Third down for the #Panthers? I wonder what will happen. #Patriots 0.39671019079976555
Wtf!! This score better change real effin quick, not taking a L from Patriots today 0.50903297116531
RT @jeffphowe: Not to get overly philosophical, but if the Patriots are the Pats, why aren’t the Panthers the Pants? 0.6401778316553294
End 3: Panthers 23, Patriots 16. Panthers have a third and 4 from their own 42 to open the 4th. 0.39671019079976555
End 3rd quarter: Panthers 23, Patriots 16 https://t.co/1B

Touchdown Amendola!!! #tombrady #dannyamendola #goraider #patriots #PatriotsvsPanthers #PatriotsNation 0.39671019079976555
This is why I love watching @Patriots games. Touchdown with three minutes left in the 4th quarter to tie it up. #CARvsNE #DoYourJob 0.8649825164137912
RT @FeitsBarstool: Every Patriots game this season has been written by the Friday Night Lights staff 0.5762747649325657
RT @FeitsBarstool: Every Patriots game this season has been written by the Friday Night Lights staff 0.5762747649325657
RT @keepTHEfaith341: No, just the Patriots. https://t.co/xH7d2k4xid 0.721293693590717
RT @PatsFans: SCORING UPDATE: Tom Brady works some magic &amp; hits Danny Amendola w/the 1yd TD.  The Patriots tie it up 30-30 w/3:09 to go in… 0.6401778316553294
Brady once again carries this team back in the game. You never can count them out when he’s around. #Patriots 0.39671019079976555
Just need a defensive stop.  What could go wrong? #Patriots 0.34581458157371153
Never count out Tom Brady a

In [22]:
df.get_values()

array([["RT @Patriots: LET'S GO. https://t.co/Ot4IjanSa1", 'Virginia',
        Timestamp('2017-10-01 16:54:59+0000', tz='UTC'),
        0.7541169002367804, 0],
       [ '@Patriots #PatsNation #NewEnglandPatriots  #DoYourJob  #sundayfootball #NFLSunday https://t.co/Pe5Oe1Qpk1',
        'Florida', Timestamp('2017-10-01 16:54:59+0000', tz='UTC'),
        0.7541169002367804, 0],
       ['RT @Patriots: #TFW you hear #boomtower is back. https://t.co/E5W2nKjnsl',
        'North Carolina', Timestamp('2017-10-01 16:55:00+0000', tz='UTC'),
        0.7541169002367804, 0],
       ..., 
       [ 'RT @amazinmikey: If Trump had been POTUS on Dec. 8, 1941, he wld have been impeached by Jan. 8, 1942. The real patriots of WWII wld never p…',
        None, Timestamp('2017-10-01 20:19:46+0000', tz='UTC'),
        0.6401778316553294, 1],
       ['@JordanStrack Bills finish 8-8 and the Patriots win the division.',
        None, Timestamp('2017-10-01 20:19:46+0000', tz='UTC'),
        0.6401778316553294, 1],

In [29]:
table = database['roll_tide']
df = tweet_processor.process_database(table)

In [31]:
from src.visualization import visualize_sentiment
from src.visualization import visualize_count
my_map = visualize_sentiment(df)
my_map